# caddy
> Caddy reverse proxy, CrowdSec security, and Cloudflare tunnel support

In [ ]:
#| default_exp caddy

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pathlib import Path
from fastcore.all import listify
from dockr.compose import Compose, service

## Caddyfile generation

`caddyfile()` generates the Caddyfile text. `caddy()` writes it and returns service kwargs for `Compose.svc()`.

In [ ]:
#| export
def caddyfile(domain, app='app', port=5001, *,
              dns=None, email=None, crowdsec=False, cloudflared=False):
    'Minimal Caddyfile for reverse-proxying app:port from domain'
    g = []
    if email: g += [f'    email {email}']
    if dns:
        p, tenv = (dns, f'{dns.upper()}_API_TOKEN') if isinstance(dns, str) else dns
        g += [f'    acme_dns {p} {{${tenv}}}']
    if crowdsec:
        g += ['    crowdsec {',
              '        api_url http://crowdsec:8080',
              '        api_key {$CROWDSEC_API_KEY}',
              '    }']
    s = []
    if crowdsec: s += ['    crowdsec']
    s += [f'    reverse_proxy {app}:{port}']
    prefix = 'http://' if cloudflared else ''
    parts = []
    if g: parts.append('{\n' + '\n'.join(g) + '\n}')
    parts.append(f'{prefix}{domain} {{\n' + '\n'.join(s) + '\n}')
    return '\n\n'.join(parts) + '\n'

In [ ]:
# Minimal
cf = caddyfile('myapp.example.com', port=5001)
assert 'myapp.example.com {' in cf
assert '    reverse_proxy app:5001' in cf
assert cf.count('{') == 1
print(cf)

In [ ]:
# With Cloudflare DNS
cf = caddyfile('myapp.example.com', port=5001, dns='cloudflare', email='me@example.com')
assert 'email me@example.com' in cf
assert 'acme_dns cloudflare {$CLOUDFLARE_API_TOKEN}' in cf
print(cf)

In [ ]:
# With CrowdSec
cf = caddyfile('myapp.example.com', port=5001, crowdsec=True)
assert 'api_url http://crowdsec:8080' in cf
assert '    crowdsec\n' in cf
assert 'api_key {$CROWDSEC_API_KEY}' in cf
print(cf)

In [ ]:
# Cloudflared mode: HTTP prefix
cf = caddyfile('myapp.example.com', port=5001, cloudflared=True)
assert cf.startswith('http://myapp.example.com {')
assert 'reverse_proxy app:5001' in cf
print(cf)

## Services

In [ ]:
#| export
_CADDY_IMG = {
    (False, False): 'caddy:2',
    (True,  False): 'serfriz/caddy-crowdsec:latest',
    (False, True):  'serfriz/caddy-cloudflare:latest',
    (True,  True):  'ghcr.io/buildplan/csdp-caddy:latest',
}

def caddy(domain, app='app', port=5001, *,
          dns=None, email=None, crowdsec=False, cloudflared=False,
          conf='Caddyfile', **kw):
    'Write Caddyfile and return Caddy service kwargs for Compose.svc()'
    Path(conf).write_text(
        caddyfile(domain, app, port, dns=dns, email=email,
                  crowdsec=crowdsec, cloudflared=cloudflared))
    img = _CADDY_IMG.get((crowdsec, dns == 'cloudflare'),
                          f'serfriz/caddy-{dns}:latest' if dns else 'caddy:2')
    env = {}
    if dns == 'cloudflare': env['CLOUDFLARE_API_TOKEN'] = '${CLOUDFLARE_API_TOKEN}'
    if dns == 'duckdns':    env['DUCKDNS_TOKEN']        = '${DUCKDNS_TOKEN}'
    if crowdsec:            env['CROWDSEC_API_KEY']      = '${CROWDSEC_API_KEY}'
    return dict(
        image=img, env=env or None,
        ports=None if cloudflared else ['80:80', '443:443', '443:443/udp'],
        volumes={f'./{conf}': '/etc/caddy/Caddyfile',
                 'caddy_data': '/data', 'caddy_config': '/config'},
        networks=['web'], depends_on=[app], restart='unless-stopped',
    ) | kw

In [ ]:
import tempfile
with tempfile.TemporaryDirectory() as tmp:
    kw = caddy('myapp.example.com', port=5001, conf=f'{tmp}/Caddyfile')
    assert kw['image'] == 'caddy:2'
    assert kw['ports'] == ['80:80', '443:443', '443:443/udp']
    assert kw['depends_on'] == ['app']
    assert 'myapp.example.com {' in Path(f'{tmp}/Caddyfile').read_text()
    print('caddy() basic OK')

with tempfile.TemporaryDirectory() as tmp:
    kw = caddy('myapp.example.com', dns='cloudflare', conf=f'{tmp}/Caddyfile')
    assert kw['image'] == 'serfriz/caddy-cloudflare:latest'
    assert kw['env'] == {'CLOUDFLARE_API_TOKEN': '${CLOUDFLARE_API_TOKEN}'}
    print('caddy() cloudflare OK')

with tempfile.TemporaryDirectory() as tmp:
    kw = caddy('myapp.example.com', crowdsec=True, conf=f'{tmp}/Caddyfile')
    assert kw['image'] == 'serfriz/caddy-crowdsec:latest'
    assert 'CROWDSEC_API_KEY' in kw['env']
    print('caddy() crowdsec OK')

with tempfile.TemporaryDirectory() as tmp:
    kw = caddy('myapp.example.com', cloudflared=True, conf=f'{tmp}/Caddyfile')
    assert kw['ports'] is None
    assert Path(f'{tmp}/Caddyfile').read_text().startswith('http://myapp.example.com {')
    print('caddy() cloudflared OK: no ports')

with tempfile.TemporaryDirectory() as tmp:
    kw = caddy('myapp.example.com', crowdsec=True, dns='cloudflare', conf=f'{tmp}/Caddyfile')
    assert kw['image'] == 'ghcr.io/buildplan/csdp-caddy:latest'
    print('caddy() crowdsec+cloudflare OK')

In [ ]:
#| export
def cloudflared_svc(token_env='CF_TUNNEL_TOKEN', **kw):
    'Cloudflare tunnel service kwargs for Compose.svc()'
    return dict(
        image='cloudflare/cloudflared:latest',
        command='tunnel --no-autoupdate run',
        env={'TUNNEL_TOKEN': f'${{{token_env}}}'},
        restart='unless-stopped',
    ) | kw

In [ ]:
kw = cloudflared_svc()
assert kw['image'] == 'cloudflare/cloudflared:latest'
assert kw['command'] == 'tunnel --no-autoupdate run'
assert kw['env'] == {'TUNNEL_TOKEN': '${CF_TUNNEL_TOKEN}'}
print('cloudflared_svc() OK')

In [ ]:
#| export
def crowdsec(collections=None, bouncer_key_env='CROWDSEC_BOUNCER_KEY', **kw):
    'CrowdSec agent service kwargs for Compose.svc()'
    cols = ' '.join(listify(collections) or [
        'crowdsecurity/linux', 'crowdsecurity/caddy', 'crowdsecurity/http-cve'])
    return dict(
        image='crowdsecurity/crowdsec:latest',
        env={'COLLECTIONS': cols, 'BOUNCER_KEY_caddy': f'${{{bouncer_key_env}}}'},
        volumes={'crowdsec-db': '/var/lib/crowdsec/data',
                 'crowdsec-config': '/etc/crowdsec'},
        networks=['web'], restart='unless-stopped',
    ) | kw

In [ ]:
kw = crowdsec()
assert kw['image'] == 'crowdsecurity/crowdsec:latest'
assert 'crowdsecurity/caddy' in kw['env']['COLLECTIONS']
assert kw['env']['BOUNCER_KEY_caddy'] == '${CROWDSEC_BOUNCER_KEY}'
assert 'crowdsec-db' in kw['volumes']
print('crowdsec() OK')

kw2 = crowdsec(collections=['crowdsecurity/linux', 'crowdsecurity/nginx'])
assert 'crowdsecurity/nginx' in kw2['env']['COLLECTIONS']
print('crowdsec() custom collections OK')

## Example: FastHTML app with Caddy

Minimal stacks — run any with `dc.save('docker-compose.yml')` then `docker compose up -d`.

In [ ]:
import tempfile
from dockr.compose import Compose

tmp = tempfile.mkdtemp()

# Stack A: Direct (Caddy auto-TLS, ports 80+443 open)
dc = (Compose()
    .svc('app', build='.', networks=['web'], restart='unless-stopped')
    .svc('caddy', **caddy('myapp.example.com', port=5001, conf=f'{tmp}/Caddyfile'))
    .network('web').volume('caddy_data').volume('caddy_config'))

d = dc.to_dict()
assert d['services']['caddy']['image'] == 'caddy:2'
assert '80:80' in d['services']['caddy']['ports']
print('=== Stack A: Direct (Caddy auto-TLS) ===')
print(dc)

In [ ]:
import tempfile
from dockr.compose import Compose

tmp = tempfile.mkdtemp()

# Stack B: cloudflared tunnel (zero open ports)
dc = (Compose()
    .svc('app', build='.', networks=['web'], restart='unless-stopped')
    .svc('caddy', **caddy('myapp.example.com', port=5001, cloudflared=True, conf=f'{tmp}/Caddyfile'))
    .svc('cloudflared', **cloudflared_svc(), networks=['web'])
    .network('web').volume('caddy_data').volume('caddy_config'))

d = dc.to_dict()
assert 'ports' not in d['services']['caddy']
assert d['services']['cloudflared']['image'] == 'cloudflare/cloudflared:latest'
print('=== Stack B: Cloudflared (zero open ports) ===')
print(dc)

In [ ]:
import tempfile
from dockr.compose import Compose

tmp = tempfile.mkdtemp()

# Stack C: CrowdSec + cloudflared (full security)
dc = (Compose()
    .svc('app', build='.', networks=['web'], restart='unless-stopped')
    .svc('caddy', **caddy('myapp.example.com', port=5001, crowdsec=True, cloudflared=True, conf=f'{tmp}/Caddyfile'))
    .svc('crowdsec', **crowdsec())
    .svc('cloudflared', **cloudflared_svc(), networks=['web'])
    .network('web')
    .volume('caddy_data').volume('caddy_config')
    .volume('crowdsec-db').volume('crowdsec-config'))

d = dc.to_dict()
assert d['services']['caddy']['image'] == 'ghcr.io/buildplan/csdp-caddy:latest'
assert d['services']['crowdsec']['image'] == 'crowdsecurity/crowdsec:latest'
print('=== Stack C: CrowdSec + cloudflared ===')
print(dc)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()